In [7]:
from bs4 import BeautifulSoup as bs
import datetime as dt
from urllib.request import urlopen

In [3]:
def date_format(d):
    d= str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    this_date = dt.date(yyyy,mm,dd)
    return this_date

In [34]:
def his_naver_index(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    # date 정리
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
    # bs 하기 및 저장
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    source = urlopen(naver_index).read()
    source = bs(source,'lxml')
    dates = source.find_all('td',class_="date")
    prices = source.find_all('td',class_="number_1")
    # 사이 값만 저장하기
    for n in range(len(dates)) :
        if dates[n].text.split('.')[0].isdigit() :
            #날짜 처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            # 사이값 저장
            if this_date <= end_date and this_date >= start_date:
                this_close = prices[n*4].text
                this_close = this_close.replace(',', '')
                this_close = float(this_close)
                #딕셔너리에 저장
                his_prices[this_date] = this_close
            elif this_date < start_date :
                return his_prices
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        his_naver_index(index_cd, start_date, end_date, page_n, last_page)   
        
    return his_naver_index  

In [37]:
index_cd = 'KPI200'
his_prices = dict()
his_naver_index(index_cd,'2019-2-1','2019-4-4')
his_prices

{datetime.date(2019, 4, 4): 286.15,
 datetime.date(2019, 4, 3): 285.65,
 datetime.date(2019, 4, 2): 281.72,
 datetime.date(2019, 4, 1): 280.32,
 datetime.date(2019, 3, 29): 276.48,
 datetime.date(2019, 3, 28): 275.08,
 datetime.date(2019, 3, 27): 277.28,
 datetime.date(2019, 3, 26): 277.6,
 datetime.date(2019, 3, 25): 277.45,
 datetime.date(2019, 3, 22): 283.13,
 datetime.date(2019, 3, 21): 282.38,
 datetime.date(2019, 3, 20): 279.92,
 datetime.date(2019, 3, 19): 279.82,
 datetime.date(2019, 3, 18): 279.93,
 datetime.date(2019, 3, 15): 279.87,
 datetime.date(2019, 3, 14): 276.64,
 datetime.date(2019, 3, 13): 276.0,
 datetime.date(2019, 3, 12): 277.73,
 datetime.date(2019, 3, 11): 275.06,
 datetime.date(2019, 3, 8): 274.84,
 datetime.date(2019, 3, 7): 278.87,
 datetime.date(2019, 3, 6): 279.85,
 datetime.date(2019, 3, 5): 280.45,
 datetime.date(2019, 3, 4): 282.51,
 datetime.date(2019, 2, 28): 283.8,
 datetime.date(2019, 2, 27): 288.76,
 datetime.date(2019, 2, 26): 287.89,
 datetime.dat

In [ ]:
#페이지 네비게이션
    if last_page == 0 :
        last_page = source.find('td', class_='pgRR').find('a')['href']
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = int(last_page)
    #다음 페이지 호출
    if page_n < last_page :
        page_n = page_n +1
        his_naver_index(index_cd, start_date, end_date, page_n, last_page)
    return his_prices